In [12]:
trainingOHCA_fileName = "generatedPoints1_100000_subzone.csv"
candidateAED_fileName = "buildings_locations_subzone.csv"
# trainingOHCA_fileName = "D:\MyDownloads\generatedPoints1_100000_subzone.csv"
# candidateAED_fileName = r"D:\MyDownloads\buildings_locations_subzone.csv"
# # testOHCA_fileName = "generatedPoints2_15000.csv"
# currentAED_fileName = "AED Details_As of 27 March 2019 (2).xlsx"

In [3]:
import pandas as pd
import numpy as np
from haversine import haversine_vector, Unit
from haversine import haversine
from os import path
import scipy.sparse
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
from pulp import *
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt     
import seaborn as sns
from itertools import combinations
from sklearn import preprocessing, cluster
from sklearn.metrics import silhouette_score
import scipy
import scipy.cluster
from matplotlib import colors as mcolors
from skimage.io import imread
import math
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
import time

In [4]:
def MCLP(S,OHCA_Cluster,AED_Cluster,d,P): #Input for MCLP to return the coordinates of AEDs to be placed
    
    # S is Distance beyond which is considered uncovered
    # P is the number of AEDs to be placed in that cluster 
    
    I = OHCA_Cluster.index.values.tolist() #Demand Node
    J = AED_Cluster.index.values.tolist() #Facility Node
    P = P
    #a = np.ones(len(OHCA_Cluster))
    
    # N is a list in which each item is a list of nodes within threshold distance of ith node
    N = [[j for j in J if d[i][j] < S] for i in I]
    #P = 2400 #Number of AEDs to be placed in this cluster
    
    #Optimization
    prob = LpProblem("MCLP",LpMaximize)
    x = LpVariable.dicts("x",J,0,1,cat="Integer")
    y = LpVariable.dicts("y",I,0,1,cat="Integer")
    #a = LpVariable.dicts("a",I,cat="Continuous")
    
    #Objective
    #prob += lpSum([a[i]*y[i] for i in I])
    prob += lpSum([y[i] for i in I])
    
    #Constraints
    for i in I:
        prob += lpSum([x[j] for j in N[i]]) >= y[i]
       # for j in N[i]:
            #if x[j] == 1:
                #prob += a[i] == 0.549*d[i][j]**-0.584
        #a[i] = [0.549*d[i][j]**-0.584 for j in N[i] if x[j] == 1]
        
    prob +=lpSum([x[j] for j in J]) == P

    #Solving
    #solver = getSolver('GUROBI')
    #prob.solve(solver)
    prob.solve(GUROBI(msg=0))
    
    x_soln = np.array([x[j].varValue for j in J])
    print ("OHCA Served is = ", value(prob.objective))
    AED_Chosen = pd.concat([AED_Cluster,pd.DataFrame(x_soln,columns=['x'])],axis=1)
    AED_Chosen = AED_Chosen[AED_Chosen['x'] == 1]
    AED_Chosen = AED_Chosen.drop(columns=['x'])
    return(AED_Chosen)

In [5]:
def kmeans_OHCA_AED(OHCA,AED,k=2): #k is the number of clusters chosen
    kmeans = KMeans(n_clusters = k, random_state = 0).fit(OHCA)
    
    OHCA_Cluster_All = pd.concat([OHCA,pd.DataFrame(kmeans.labels_,columns = ["cluster"])],axis=1)
    AED_lable = kmeans.predict(AED)
#     print(len(AED_lable))
    AED_Cluster_All = pd.concat([AED, pd.DataFrame(AED_lable,columns = ["cluster"])], axis=1)

    return OHCA_Cluster_All, AED_Cluster_All

In [6]:
# input: two clusters 
# output: All_AED_in_Cluster
def MCLP_in_Cluster(OHCA1,candidateAED1,p_subzone,k=2):
    OHCA1_clusters, candidateAED1_clusters = kmeans_OHCA_AED(OHCA1, candidateAED1,k)
    All_AED_in_Cluster=pd.DataFrame(columns=['LatAED', 'LongAED']) 
    for i in range(k):
        OHCA20a = OHCA1_clusters[OHCA1_clusters['cluster']==i][["LatOHCA","LongOHCA"]].reset_index(drop=True)
        candidateAED20a = candidateAED1_clusters[candidateAED1_clusters['cluster']==i][["LatAED","LongAED"]].reset_index(drop=True)
        if len(candidateAED20a)>0:
            d1 = haverDist(OHCA20a,candidateAED20a)
            sub_AED_df = MCLP(100,OHCA20a,candidateAED20a,d1,int(p_subzone/k))
            All_AED_in_Cluster=All_AED_in_Cluster.append(sub_AED_df)
        else: pass
    return All_AED_in_Cluster

In [7]:
def coveragefunction(OHCA_Cluster,AED_Cluster):
    pij = haversine_vector(list(zip(OHCA_Cluster.LatOHCA, OHCA_Cluster.LongOHCA)), list(zip(AED_Cluster.LatAED, AED_Cluster.LongAED)), Unit.METERS, comb=True).T
    pij = [[1 if p <= 20 else p for p in s] for s in pij]
    pij = [[np.exp(-0.05*(p - 20)) if p > 20 and p <= 100 else p for p in s] for s in pij]
    pij = [[0 if p>100 else p for p in s] for s in pij]
    return(pij)

In [8]:
def haverDist(OHCA_Cluster,AED_Cluster):
    d = haversine_vector(list(zip(OHCA_Cluster.LatOHCA, OHCA_Cluster.LongOHCA)), list(zip(AED_Cluster.LatAED, AED_Cluster.LongAED)), Unit.METERS, comb=True).T
    return(d)

In [13]:
#test_OHCA = pd.read_csv(testOHCA_fileName, names = ["LatOHCA", "LongOHCA"], header = None)
OHCA = pd.read_csv(trainingOHCA_fileName)
candidateAED = pd.read_csv(candidateAED_fileName)
# candidateAED.columns = ["LatAED","LongAED"]
P_count=OHCA['region'].value_counts()
len(P_count.values)
P_count.values.sum()
P_count


300    1966
273    1960
179    1646
105    1435
141    1428
       ... 
153      18
69       16
323      14
4         2
259       1
Name: region, Length: 327, dtype: int64

In [14]:
P_count2=candidateAED['region'].value_counts()
print(P_count2)

141    7122
164    5056
249    3933
162    3695
150    3694
       ... 
211       1
280       1
258       1
272       1
221       1
Name: region, Length: 321, dtype: int64


In [15]:
for subzone in range(1,333):
#     if subzone != 7 and subzone!=14 and subzone != 48 : 
#     if subzone not in (7, 8, 9,159,304):
    if subzone not in (7, 8, 9,159,174,231, 239,283, 284, 287,301, 304, 305, 317, 323):
#         p_subzone=int(P_count.at[subzone]*9800/96228)
        print(subzone, P_count2.at[subzone] )

1 6
2 179
3 100
4 167
5 78
6 3
10 604
11 38
12 83
13 8
14 263
15 113
16 45
17 313
18 45
19 27
20 75
21 58
22 162
23 747
24 60
25 70
26 88
27 68
28 2856
29 12
30 91
31 106
32 315
33 26
34 127
35 20
36 18
37 81
38 789
39 71
40 95
41 215
42 146
43 65
44 630
45 43
46 115
47 31
48 93
49 37
50 75
51 283
52 140
53 57
54 198
55 480
56 412
57 191
58 28
59 52
60 218
61 105
62 144
63 99
64 124
65 2531
66 142
67 10
68 1381
69 93
70 63
71 62
72 428
73 153
74 211
75 150
76 135
77 79
78 252
79 200
80 25
81 13
82 253
83 292
84 136
85 157
86 720
87 864
88 121
89 198
90 7
91 32
92 176
93 46
94 299
95 292
96 293
97 31
98 131
99 1510
100 2840
101 1138
102 386
103 878
104 48
105 1887
106 215
107 75
108 1609
109 391
110 1478
111 136
112 331
113 77
114 78
115 334
116 11
117 460
118 70
119 84
120 58
121 45
122 62
123 1618
124 113
125 233
126 89
127 436
128 885
129 242
130 129
131 1110
132 363
133 940
134 625
135 217
136 89
137 289
138 575
139 1047
140 53
141 7122
142 153
143 1884
144 2663
145 112
146 157
147 

In [16]:
ALL_AED= pd.DataFrame(columns=['LatAED', 'LongAED']) 

In [17]:
%%time

start_time = time.time()

ALL_AED= pd.DataFrame(columns=['LatAED', 'LongAED']) 
all_AED_num=9880
all_OHCA_num=96228

for subzone in range(1,333):
#     if subzone not in (7, 8, 9,159,304):
    if subzone not in (7, 8, 9,159,174,231, 239,283, 284, 287,301, 304, 305, 317, 323):
        p_subzone=int(P_count.at[subzone]*all_AED_num/all_OHCA_num)
        print(subzone, P_count.at[subzone], p_subzone )
        OHCA1=OHCA[OHCA['region']== subzone]
        OHCA1=OHCA1[['LatOHCA','LongOHCA']].reset_index(drop=True)
        candidateAED1 = candidateAED[candidateAED['region']==subzone].copy()
        candidateAED1 = candidateAED1[['LatAED','LongAED']].reset_index(drop=True)
        candidateAED1 = candidateAED1.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
        d1 = haverDist(OHCA1,candidateAED1)
        sub_AED_df = MCLP(100,OHCA1,candidateAED1,d1,p_subzone)
        ALL_AED=ALL_AED.append(sub_AED_df)
        
        
print("--- %s seconds ---" % (time.time() - start_time))

1 219 22
Academic license - for non-commercial use only - expires 2021-10-09
Using license file /Users/wanbing/gurobi.lic
OHCA Served is =  None
2 153 15
OHCA Served is =  136.0
3 199 20
OHCA Served is =  186.0
4 2 0
OHCA Served is =  0.0
5 151 15
OHCA Served is =  109.0
6 153 15
OHCA Served is =  None
10 179 18
OHCA Served is =  55.0
11 523 53
OHCA Served is =  None
12 29 2
OHCA Served is =  20.0
13 117 12
OHCA Served is =  None
14 507 52
OHCA Served is =  274.0
15 186 19
OHCA Served is =  164.0
16 46 4
OHCA Served is =  40.0
17 217 22
OHCA Served is =  197.0
18 102 10
OHCA Served is =  92.0
19 23 2
OHCA Served is =  20.0
20 53 5
OHCA Served is =  45.0
21 155 15
OHCA Served is =  142.0
22 411 42
OHCA Served is =  272.0
23 94 9
OHCA Served is =  57.0
24 236 24
OHCA Served is =  120.0
25 232 23
OHCA Served is =  190.0
26 161 16
OHCA Served is =  141.0
27 175 17
OHCA Served is =  162.0
28 215 22
OHCA Served is =  215.0
29 18 1
OHCA Served is =  10.0
30 54 5
OHCA Served is =  45.0
31 45 4

OHCA Served is =  575.0
250 400 41
OHCA Served is =  240.0
251 465 47
OHCA Served is =  207.0
252 192 19
OHCA Served is =  121.0
253 242 24
OHCA Served is =  131.0
254 369 37
OHCA Served is =  191.0
255 196 20
OHCA Served is =  100.0
256 225 23
OHCA Served is =  188.0
257 503 51
OHCA Served is =  470.0
258 154 15
OHCA Served is =  None
259 1 0
OHCA Served is =  0.0
260 446 45
OHCA Served is =  425.0
261 429 44
OHCA Served is =  377.0
262 207 21
OHCA Served is =  51.0
263 511 52
OHCA Served is =  389.0
264 301 30
OHCA Served is =  235.0
265 138 14
OHCA Served is =  97.0
266 495 50
OHCA Served is =  None
267 244 25
OHCA Served is =  184.0
268 248 25
OHCA Served is =  178.0
269 602 61
OHCA Served is =  124.0
270 126 12
OHCA Served is =  32.0
271 147 15
OHCA Served is =  83.0
272 112 11
OHCA Served is =  None
273 1960 201
OHCA Served is =  None
274 71 7
OHCA Served is =  31.0
275 49 5
OHCA Served is =  10.0
276 100 10
OHCA Served is =  45.0
277 93 9
OHCA Served is =  91.0
278 72 7
OHCA Ser

In [ ]:
# --- 146.1881809234619 seconds ---
# Wall time: 2min 26s

In [19]:
print('The overall MCLP AED result including all subzone is in this dataframe, you can use it to further calculate metrics : ')
ALL_AED

The overall MCLP AED result including all subzone is in this dataframe, you can use it to further calculate metrics : 


,LatAED,LongAED
8,1.294223,103.835815
35,1.294716,103.840051
37,1.295553,103.840951
55,1.295497,103.834969
58,1.294718,103.834202
...,...,...
953,1.349277,103.844387
1004,1.348914,103.845762
1052,1.351166,103.847296
102,1.277594,103.637965


In [20]:
def totalCoverage(ohca_df, aed_df, MAX_DIST_METERS = 100):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000
    
    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 100, 'isCovered'] = 'covered'
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] > 100,'isCovered'] = 'uncovered'
    # each OHCA having an AED within 100m has isCovered = 'covered', otherwise 'uncovered'
    #print(cp_ohca)
    
    
    within_100m  = sum(cp_ohca['isCovered'] == 'covered')
    outside_100m = len(cp_ohca) - within_100m
    total_coverage_within_100m = within_100m / (within_100m + outside_100m)
    print("total coverage = ", total_coverage_within_100m)
    
    return total_coverage_within_100m

def partial_coverage(ohca_df, aed_df, ALPHA = 0.05):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000

    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 20,'p_ij_coverage'] = 1.0
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] >= 100,'p_ij_coverage'] = 0.0
    cp_ohca.loc[(cp_ohca['dist_nearest_AED'] < 100) & (cp_ohca['dist_nearest_AED'] > 20),
                'p_ij_coverage'] = np.exp(- ALPHA * (cp_ohca['dist_nearest_AED'] - 20))
    # p_ij of each OHCA calulated
    # print(cp_ohca)
    
    return cp_ohca['p_ij_coverage'].mean()

def expectedsurvival(OHCA,AED):
    prob = 0
    for i in range(len(OHCA)):
        survival = haversine_vector((OHCA.LatOHCA[i], OHCA.LongOHCA[i]), list(zip(AED.LatAED, AED.LongAED)), Unit.METERS, comb=True).T[0]
        survival = survival/(6.15*1000/60)*2
        survival = [p if np.isnan(p) == False else 0 for p in survival]
        survival = [1 if p<=1 and p != 0 else p for p in survival]
        survival = [p if p<20 else 0 for p in survival]
        survival = [p if p==0 or p == 1 else ((p**-0.584)*0.549) for p in survival]
        prob += max(survival)
    expectedSurvival = prob/len(OHCA)
    return(expectedSurvival)

In [21]:
def average_distance(ohca_df, aed_df):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000

    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    
    return cp_ohca['dist_nearest_AED'].mean()

In [22]:
AED_Chosen_All_MCLP=ALL_AED.reset_index(drop=True)
#AED_Chosen_All_MCLP = pd.read_csv("MCLP_SingleBest_Output.csv",index_col=0)
#total coverage 
MCLP_TC = totalCoverage(OHCA,AED_Chosen_All_MCLP,100)
print("MCLP total coverage is : ", MCLP_TC)
#partial coverage
MCLP_PC = partial_coverage(OHCA,AED_Chosen_All_MCLP)
print("MCLP partial coverage is : ", MCLP_PC)
#expected survival
MCLP_ES = expectedsurvival(OHCA,AED_Chosen_All_MCLP)
print("MCLP expected survival is : ", MCLP_ES)
#average distance to nearest AED
MCLP_AD = average_distance(OHCA, AED_Chosen_All_MCLP)
print("MCLP average distance is : ", MCLP_AD)

total coverage =  0.6783160826370703
MCLP total coverage is :  0.6783160826370703
MCLP partial coverage is :  0.15087424260977958
MCLP expected survival is :  0.4917124246355892
MCLP average distance is :  162.31513985605798
